In [5]:
import json
import keyword
import unicodedata
from collections import defaultdict
import pandas as pd
from extract import SocialETL, construct_query_for_twarc, extract_tags, SocialDB, UserETL
import hashtags as h
import hashtags_readcsv as r_csv
from rich import print
from rich.console import Console
from rich.progress import track
from rich.table import Table
#import warnings 
#warnings.filterwarnings('always')   #suppresses all warnings
#warnings.filterwarnings('ignore')

import unicodedata
from collections import defaultdict
from pathlib import Path

import pandas as pd
from extract import extract_tags
from rich import print
from rich.progress import track

## 1 parte: categorizzazione hashtags per prorus, proukr, pax

In [2]:
def ensure_latin(s): 
    return (
        unicodedata.normalize("NFKD", s).encode("latin-1", "ignore").decode("latin-1")
    )


def construct_query_for_pandas(root_tags: list) -> str:
    my_query = " or ".join(root_tags)
    return my_query


def create_score(df: pd.DataFrame, one_hashtag: str, root_tags: dict) -> list:
    # takes a df of all tweets, each row is a tweet, each column is
    # a hashtag either [True | False] depending on whether the hashtag
    # is in the tweet. Takes one_hashtag, the hashtag to score, and
    # root_tags, a dict of {category: [hashtags]}
    # TODO: make it so it accepts a list of {category: [hashtags]} instead of just one
    threshold_support = 0.5 * len(df) / 10000  # set the threshold_support here!!

    if threshold_support < 1:
        threshold_support = 1
    # print(f"Looking for {one_hashtag} with support {threshold_support}…")
    mask = df[one_hashtag]
    df = df[mask]
    # print(f"I found {len(df)} tweets with {one_hashtag}")
    if len(df) < threshold_support:
        return False
    """for col in df:
        if df[col].any():
            print(f"We have {col} hashtag")"""
    results = []
    # START OF TESTING:
    """    print(df.info())
    print(df.describe())
    print(df.head())
    print(f"[purple]Dude, here's the breakdown:")
    for tag in root_tags.values():
        print(f"[blue]This is {tag[0]}:")
        print(df[tag[0]])"""
    for category, tag_madre in root_tags.items():
        try:
            my_query = construct_query_for_pandas(tag_madre)
            # print(f"The pandas query is: [red]{my_query}")
            temp_df = df.query(my_query)
            # print(f"Holy jezuz: {len(temp_df)} and {len(df)}")
            results.append(round(len(temp_df) / len(df), 4))
        except KeyError:
            results.append(0)

    # print(results)
    return (results, len(df))


def do_search(tagmadre, pages):
    # construct the initial query to Twarc
    query_madre = construct_query_for_twarc(tagmadre)
    m = SocialETL(
        query=f"({query_madre})",
        pages=pages,
        recent=False,
    )

    # dropping any tweets with no hashtags (I think)
    tweets_with_hashtag = m.df[["id", "entities.hashtags"]].dropna()
    print(
        f"{len(m.df)} tweets retrieved\nwith query '{query_madre}'\nof which {len(tweets_with_hashtag)} tweets with at least 1 hashtag."
    )

    # evaluate the string in "entities.hashtags" to an actual list of dicts
    tweets_with_hashtag["entities.hashtags"] = tweets_with_hashtag[
        "entities.hashtags"
    ].map(eval)

    # make a simple list of strings, one hashtag is one string, into column "tags"
    tweets_with_hashtag["tags"] = tweets_with_hashtag["entities.hashtags"].map(
        extract_tags
    )
    tweets_with_hashtag = tweets_with_hashtag.drop(columns="entities.hashtags")

    # hashtags: EXPLODE *musica dei power ranger*
    all_hashtags = set(tweets_with_hashtag["tags"].explode())

    # keyword level: eliminate all python keywords from columns because otherwise we are in no man's land
    keywordsss = keyword.kwlist
    keywordsss.extend(keyword.softkwlist)
    all_hashtags = all_hashtags.difference(keywordsss)
    all_hashtags = all_hashtags.difference(set(("",)))

    print(f"We have {len(all_hashtags)} unique hashtags.")

    tweets_with_hashtag.set_index("id", inplace=True)

    col_h = sorted(list(all_hashtags))
    df_h = pd.DataFrame(columns=col_h)
    tweets_with_hashtag = pd.concat([tweets_with_hashtag, df_h], axis=1)
    tweets_with_hashtag = tweets_with_hashtag.fillna(False)

    def assign_hashtag_to_tweet(row: pd.Series) -> pd.Series:
        for tag in row["tags"]:
            if tag in all_hashtags:
                row.loc[tag] = True
        return row

    tweets_with_hashtag = tweets_with_hashtag.apply(assign_hashtag_to_tweet, axis=1)
    tweets_with_hashtag = tweets_with_hashtag.drop(columns=["tags"])

    for madre in tagmadre.values():
        print(f"[red]Let's write the describe of column '{madre[0]}'")
        print(tweets_with_hashtag[madre[0]].describe())

    tweets_with_hashtag.to_csv(f"twee_hash_{pages}.csv")
    print("[green]All done.")


if __name__ == "__main__":
    # params
    top_results_to_take = 3
    pages_to_do = 800

    # set the initial "parent" hashtags for each category
    tag_madre = {
        "proukr": ["slavaukraini"],
        "prorus": ["istandwithputin"],
        "pax": ["stopwarinukraine"],
    }

    # code
    # first run
    do_search(tag_madre, pages_to_do)


Output()

rate limit exceeded: sleeping 110.18333053588867 secs
rate limit exceeded: sleeping 124.55334281921387 secs


In [7]:
df=pd.read_csv('twee_hash_800.csv')

In [9]:
df.shape

(79426, 7637)

In [8]:
df.head(10)

,Unnamed: 0,08cykla,1000ai,100millionsdemorts,101st,10percentforthebigguy,13luglio,14juillet,14juillet2022,15july2022,...,znazis,zolkin,zombies,zsu,zucker,zurich,zusammenhalten,zustimmung,zwar,zwastika
0,1555959736935251969,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1555959615136976897,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1555959613626925057,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1555959517011116033,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1555959220692033537,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,1555959209031860227,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,1555959146658369539,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,1555959123124043782,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,1555959043314925568,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,1555959032392867842,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
def ensure_latin(s):
    return (
        unicodedata.normalize("NFKD", s).encode("latin-1", "ignore").decode("latin-1")
    )


def construct_query_for_pandas(root_tags: list) -> str:
    my_query = " or ".join(root_tags)
    return my_query


def create_score(df: pd.DataFrame, one_hashtag: str, root_tags: dict) -> list:
    # takes a df of all tweets, each row is a tweet, each column is
    # a hashtag either [True | False] depending on whether the hashtag
    # is in the tweet. Takes one_hashtag, the hashtag to score, and
    # root_tags, a dict of {category: [hashtags]}
    # TODO: make it so it accepts a list of {category: [hashtags]} instead of just one
    threshold_support = 0.5 * len(df) / 10000  # set the threshold_support here!!

    if threshold_support < 1:
        threshold_support = 1
    # print(f"Looking for {one_hashtag} with support {threshold_support}…")
    mask = df[one_hashtag]
    df = df[mask]
    # print(f"I found {len(df)} tweets with {one_hashtag}")
    if len(df) < threshold_support:
        return False
    """for col in df:
        if df[col].any():
            print(f"We have {col} hashtag")"""
    results = []
    # START OF TESTING:
    """    print(df.info())
    print(df.describe())
    print(df.head())
    print(f"[purple]Dude, here's the breakdown:")
    for tag in root_tags.values():
        print(f"[blue]This is {tag[0]}:")
        print(df[tag[0]])"""
    for category, tag_madre in root_tags.items():
        try:
            my_query = construct_query_for_pandas(tag_madre)
            # print(f"The pandas query is: [red]{my_query}")
            temp_df = df.query(my_query)
            # print(f"Holy jezuz: {len(temp_df)} and {len(df)}")
            results.append(round(len(temp_df) / len(df), 4))
        except KeyError:
            results.append(0)

    # print(results)
    return (results, len(df))


def do_search(tagmadre, pages):
    # construct the initial query to Twarc
    tweets_with_hashtag = pd.read_csv(Path("twee_hash_800.csv"), index_col=0)

    # hashtags: EXPLODE *musica dei power ranger*
    all_hashtags = set(tweets_with_hashtag.columns)

    # keyword level: eliminate all python keywords from columns because otherwise we are in no man's land
    keywordsss = keyword.kwlist
    keywordsss.extend(keyword.softkwlist)
    all_hashtags = all_hashtags.difference(keywordsss)
    all_hashtags = all_hashtags.difference(set(("",)))

    # up to here we have only dealt with a dataframe of tweets. Now we switch to dataframe of hashtags

    # create a dataframe with all hashtags and their scores
    all_hashtags_as_dict = {}
    supp = {}
    for hashtag in track(all_hashtags):
        # discard based on support
        pass
        # calculate scores only on hashtags with enough support
        score = create_score(tweets_with_hashtag, hashtag, tagmadre)
        if score is not False:
            # score[0] is the scores, score[1] is the support
            all_hashtags_as_dict[hashtag] = score[0]
            supp[hashtag] = score[1]
    all_hashtags_df = pd.DataFrame.from_dict(
        all_hashtags_as_dict, orient="index", columns=tagmadre
    )

    # now, "categorize" hashtags. The hashtag gets the category of its max score,
    # as long as it is > `threshold_certainty`
    # TODO: remember that we still need to account for hashtag support
    # (i.e. number of tweets supporting that hashtag)
    threshold_certainty = 0.5
    tags_categorized = defaultdict(list)
    for hashtag, scores in all_hashtags_df.iterrows():
        # print(hashtag, scores)  # TODO: check if the scores are different enough among 3 categories
        # print(scores.idxmax(), scores.max())
        if scores.max() > threshold_certainty:
            tags_categorized[scores.idxmax()].append((hashtag, scores.max()))
            other_scores = set(scores).difference(set((scores.max(),)))
            for sco in other_scores:
                if scores.max() < 1.2 * sco:
                    print(f"Attention. For {hashtag} the scores are\n{scores}")

    # tags_categorized.sort(key=lambda x: x[1], reverse=True)
    """    tags_categorized = {
        k: sorted(v, key=lambda item: item[1], reverse=True)
        for k, v in tags_categorized.items()
    }"""

    return tags_categorized, supp


if __name__ == "__main__":
    # params
    top_results_to_take = 3
    pages_to_do = 300

    # set the initial "parent" hashtags for each category
    tag_madre = {
        "proukr": ["slavaukraini"],
        "prorus": ["istandwithputin"],
        "pax": ["stopwarinukraine"],
    }

    # code
    # first run
    end_results, tags_support = do_search(tag_madre, pages_to_do)

    # second run
    """
    end_results = {
        k: [x[0] for x in v[:top_results_to_take]] for k, v in end_results.items()
    }
    print("miao", end_results)
    end_results = do_search(end_results)
    """

    with open(f"hashtags_{pages_to_do}.json", "w", encoding="utf-8") as f:
        json.dump(end_results, f, ensure_ascii=False, indent=4)
    with open(f"supports_{pages_to_do}.json", "w", encoding="utf-8") as f:
        json.dump(tags_support, f, ensure_ascii=False, indent=4)


## 2 parte: scaricare df e distinzione 4 cat (no care) dalle altre

In [8]:
with open('hashtags_300.json', 'r') as f:
    my_dict = json.load(f)

In [9]:
my_dict

{'proukr': [['novoberyslav', 1.0],
  ['ukrainy', 1.0],
  ['energy', 1.0],
  ['kolonisaatio', 1.0],
  ['socialmediatroll', 1.0],
  ['fuckrussia', 0.9661],
  ['learn', 1.0],
  ['inguletz', 1.0],
  ['skadovsk', 0.7333],
  ['tiktok', 0.9474],
  ['berlin', 0.8261],
  ['bringbackboris', 1.0],
  ['potus', 1.0],
  ['openhumanecorridors', 1.0],
  ['ddosecrets', 1.0],
  ['ukrainemustwin', 1.0],
  ['drinkforpeace', 1.0],
  ['squad3o3', 1.0],
  ['itsecurity', 1.0],
  ['crimeanbridge', 1.0],
  ['wearenafo', 1.0],
  ['victoryforukraine', 1.0],
  ['braveukraine', 0.8491],
  ['nazisti', 0.9],
  ['burjatien', 1.0],
  ['google', 1.0],
  ['blm', 0.9886],
  ['russland', 0.9243],
  ['putinterorist', 0.6],
  ['ualayka', 1.0],
  ['fckputine', 1.0],
  ['soldiers', 1.0],
  ['dynamokiev', 0.6],
  ['transnistria', 1.0],
  ['helmets', 1.0],
  ['vdv', 1.0],
  ['anonymiss', 1.0],
  ['sendukrainehimars', 1.0],
  ['infosec', 1.0],
  ['ssu', 1.0],
  ['melitopol', 0.9636],
  ['boxing', 1.0],
  ['motherrussia', 0.75],
 

In [2]:
#modifiche: cambiare data e togliere lang eng
m = SocialETL(query="(#eurovision OR #covid OR #pridemonth OR #grammys OR #justiceforjohnnydepp OR #oscars OR #elonmusk OR #putin OR #russia OR #onepiece OR #ukraine OR #war OR #strangerthings4)", pages=100, recent=False) #strangerthings4 

Output()

In [4]:
m.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 74 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     100 non-null    object 
 1   conversation_id                        100 non-null    object 
 2   referenced_tweets.replied_to.id        1 non-null      object 
 3   referenced_tweets.retweeted.id         83 non-null     object 
 4   referenced_tweets.quoted.id            6 non-null      object 
 5   author_id                              100 non-null    object 
 6   in_reply_to_user_id                    1 non-null      object 
 7   retweeted_user_id                      83 non-null     object 
 8   quoted_user_id                         6 non-null      object 
 9   created_at                             100 non-null    object 
 10  text                                   100 non-null    object 
 11  lang   

In [3]:
user_id=m.df['author_id']
set_user=set(user_id)
#un utente può aver fatto più di un tweet, in questo modo andiamo a vedere i suoi ultimi tweet solo una volta.
print(len(set_user))

In [4]:
#estarre pure retweeted_user_id per edge
hashtag_for_user={}
tweet_for_user={}
retweet_for_user={}
verified_for_user={}
location_for_user={}
#name_for_user={}
#username_for_user={}

for e in set_user:                                                             #1 pages=20 tweet   
    u=UserETL(e,pages=5)                                                       #aggiungere più pagine (1pag=20)
    #preprocessing hashtag
    u.df=u.df.dropna(subset=['entities.hashtags'])                             #drop nan values on column hashtags
    u.df.reset_index()
    u.df['entities.hashtags']=u.df['entities.hashtags'].apply(eval)            #convert into dict
    u.df["entities.hashtags"] = u.df["entities.hashtags"].map(h.extract_tags)  #extract tags
    hashtag_for_user[e]=u.df["entities.hashtags"].tolist()                     #convert pandas series into a list
    #preprocessing retweet
    u.df=u.df.dropna(subset=['retweeted_user_id'])
    retweet_for_user[e]=u.df['retweeted_user_id'].tolist()
    tweet_for_user[e]=u.df['id'].tolist()
    #saving other attributes (to assigne it to a nodes as property)
    verified_for_user[e]=u.df['author.verified']
    location_for_user[e]=u.df['author.location'].tolist()
   # name_for_user[e]=u.df['author.name']
   # username_for_user[e]=u.df['author.username']
    
    
    

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

UnboundLocalError: local variable 'df' referenced before assignment

In [9]:
user_id=[]
tweet_id=[]
hashtag=[]
retweet=[]
location=[]
verified=[]
for u,h in hashtag_for_user.items():
    user_id.append(u)
    hashtag.append(h)
for u,t in tweet_for_user.items():
    tweet_id.append(t)
for u,r in retweet_for_user.items():
    retweet.append(r)
for u,l in location_for_user.items():
    location.append(l)
for u,v in verified_for_user.items():
    verified.append(v)

In [30]:
#create a pandas dataframe
df_nodes=pd.DataFrame({"user_id":user_id,
                       'tweet_id':tweet_id,
                      'retweet_id': retweet,
                      "hashtag":hashtag,
                     'location':location,
                     'verified':verified})
df_nodes

,user_id,tweet_id,retweet_id,hashtag,location,verified
0,632276693,"[1556006364446695426, 1552334974904217601, 153...","[1541086591229231104, 1290441690, 62053208, 97...","[[onepiece, onepiecefilmred], [dbacks], [dback...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0 False 5 False 45 False 49 Fals...
1,2386380738,"[1556006077413871616, 1556005694352261120, 155...","[1131816228274692096, 188233441, 198949082, 19...","[[], [, ], [, ], [, , sabakan, , ], [op_filmre...","[DISH//だし！放送地域𓌉◯𓇋 ‎, DISH//だし！放送地域𓌉◯𓇋 ‎, DISH/...",0 False 1 False 2 False 3 Fals...
2,731210404823740417,"[1556010727609081858, 1556010704930578433, 155...","[606254342, 606254342, 1891490382, 1891490382,...","[[mariupol], [ukraina], [russian, nikopol, kry...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0 False 1 False 8 False 9 Fals...
3,314157950,"[1556003758617907210, 1556003704419008512, 155...","[298677835, 1475688399801569281, 1484110071902...","[[cumex], [nationalhymne], [ichhabemitgemacht]...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0 False 2 False 6 False 12 Fals...
4,27542573,"[1556009144418705409, 1556009107127242752, 155...","[1373371839062347785, 1373371839062347785, 137...","[[gazaunderattack], [gazaunderattack], [gazaun...","[São Paulo, Brazil, São Paulo, Brazil, São Pau...",0 False 1 False 2 False 3 Fals...
...,...,...,...,...,...,...
4433,793071405965475841,"[1556007375416467456, 1556007257766338560, 155...","[1295405245011460096, 520690173, 605126134, 24...","[[breaking, gaza, palestine], [gazaunderattack...","[England, United Kingdom, England, United King...",2 False 3 False 7 False 10 Fals...
4434,2275138549,"[1556003709657686016, 1555978549554810882, 155...","[1495480590572961792, 934810674450059265, 9348...","[[ukraine], [ukrainerussiawar], [zaporizhzhia,...","[Ucraina, Ucraina, Ucraina, Ucraina, Ucraina, ...",0 False 2 False 11 False 35 Fals...
4435,1157318905930457090,"[1556010797603516416, 1555866823043690498, 155...","[198949082, 1474276996394123266, 2656582075, 2...","[[imax, op_filmred, onepiece, shanks, ], [], [...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",0 False 1 False 2 False 3 Fals...
4436,1471423949904957441,"[1556013734836977667, 1556012397713833987, 155...","[308323017, 1488444267223830533, 1519446270, 1...","[[hongrie, gyor], [taiwan, ukraine], [liberte,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",1 False 3 False 4 False 10 Fals...


In [55]:
#df_nodes.to_csv('4438_users_csv')
df_nodes=pd.read_csv('4438_users_csv')

In [56]:
#convert nested lists on set
outer_list = set() 
def lists_to_list(nested_lists): 
    for el in nested_lists: 
        #outer_list = set()
        if type(el) == list: 
            outer_list.add([el]) 
        #else: 
     #   outer_list.add(el)
    return set(outer_list)

for col in df_nodes:
    outer_list = set() 
    df_nodes[col]=df_nodes[col].map(lists_to_list)
df_nodes

In [57]:
outer_list = set()
df_nodes['hashtag']=df_nodes['hashtag'].map(lists_to_list)
outer_list = set()
df_nodes['tweet_id']=df_nodes['tweet_id'].map(lists_to_list)
outer_list = set()
df_nodes['retweet_id']=df_nodes['retweet_id'].map(lists_to_list)
outer_list = set() 
df_nodes['location']=df_nodes['location'].map(lists_to_list)
outer_list = set() 
df_nodes['verified']=df_nodes['verified'].map(lists_to_list)

In [58]:
df_nodes

,Unnamed: 0,user_id,tweet_id,retweet_id,hashtag,location,verified
0,0,632276693,{},{},{},{},{}
1,1,2386380738,{},{},{},{},{}
2,2,731210404823740417,{},{},{},{},{}
3,3,314157950,{},{},{},{},{}
4,4,27542573,{},{},{},{},{}
...,...,...,...,...,...,...,...
4433,4433,793071405965475841,{},{},{},{},{}
4434,4434,2275138549,{},{},{},{},{}
4435,4435,1157318905930457090,{},{},{},{},{}
4436,4436,1471423949904957441,{},{},{},{},{}


In [ ]:
df_nodes.to_csv('set_4438_users_csv')
df_nodes=pd.read_csv('set_4438_users_csv')

In [ ]:
proukr=set(["slavaukraini"]) #1 cat
prorus=set(["istandwithputin", "istandwithrussia"]) #2 cat
pax=set(["stopwarinukraine"]) #3 cat
cat=set.union(proukr,prorus,pax)
count1=0
#count2=0
#count3=0
count4=0
index1=[]
#index2=[]
#index3=[]
index4=[]

for i, row in enumerate(df_nodes['hashtag']): 
    if row.intersection(cat)==set():#no intersezione   
        #print(row.intersectionn(cat))
        #print("l'utente {}  appartiene alla 4 categoria".format(row.intersection(cat)))
        #print("4 categoria", row)
        count4 += 1
        index4.append(i)
        
    else: #c'è intersezione: uno della 3 categorie
        #print("non è la 4 categoria perchè contiene",row.intersection(cat))
       # if row.intersection(proukr)!=set(): #c'è intersez con proukr
            count1 += 1
            index1.append(i) #aggiungo alle rige da tgliere
            print("l'utente {} non  appartiene alla 4 categoria perchè usa {}".format(i,row.intersection(cat)))
       # elif row.intersection(prorus)!=set():
          #  count2 += 1
           # index2.append(i)
        #elif row.intersection(pax)!=set():
           # count3 += 1
           #index3.append(i)

In [ ]:
print('nocare = {} \n'
      'altre categorie = {}'.format(count4, count1))

## 3 parte categorizzazione tweet e utenti di proukr, prorus, pax

In [ ]:
parte fabio

## 4 parte: edge tra utenti nel dataset

In [ ]:
edges=[]
for i,t in enumerate(df_nodes['tweet_id']):
    for i_r,r in enumerate(df_nodes['retweet_id']):
        if t.intersection(r)!=set():
            edges.append({
                'from': df_nodes['user_id'].loc[i],
                'to': df_nodes['user_id'].loc[i_r]
            })
print(edges)

In [ ]:
print(len)

# 5 Creazione rete

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:
#assegnazione propietà al nodo, ex: categoria, 'author.location', 'author.verified', 
for i, e in enumerate(df_nodes['id']):
    G.add_node(e) # cat=df_nodes['cat'][i] ->aggiungere cat a ogni nodo
list(G.nodes.data()) #we can use all the operation about dictionary on these attributes

In [ ]:
#assegnazione link tra nodi
for e in edges:
    for k,v in e.items():
        if k=="from":
            v0=v
        if k=='to':
            v1=v
        G.add_edge(v0,v1)

In [ ]:
G.number_of_edges()

In [ ]:
G.number_of_nodes()

In [ ]:
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
nx.draw(G)